In [46]:
from selenium import webdriver
import pandas as pd
import numpy as np

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [10]:
PATH = "/usr/local/bin/chromedriver"

In [11]:
service = Service(PATH)
driver = webdriver.Chrome(service=service)

In [35]:
driver.get("https://www.imdb.com/search/title/?groups=top_1000&count=100&sort=user_rating,desc")

In [36]:
soup = BeautifulSoup(driver.page_source, 'html.parser')


In [53]:
title = []
years = []
duration = []
classifications = []
ratings = []
vote_count = []

metascores = []
description = []

In [54]:
# update movie title 

movie_title = soup.find_all('a', attrs={'class':'ipc-title-link-wrapper'})
for store in movie_title:
    name = store.h3.text
    title.append(name)


In [55]:
# update movie year, duration, classification

details = soup.findAll('div', attrs = {'class':'sc-b189961a-7 feoqjK dli-title-metadata'})

for detail in details:
    year_span = detail.span
    duration_span = year_span.find_next_sibling('span')
    class_span = duration_span.find_next_sibling('span')
    
    years.append(year_span.text)
    duration.append(duration_span.text)
    classifications.append(class_span.text)

In [56]:
# update movie rating, vote count

stars = soup.findAll('span', attrs={'class':'ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating'})

stars = [star.text for star in stars]

for star in stars:
    rating, vote = star.split('\xa0')
    ratings.append(rating)
    vote_count.append(vote.strip('()'))

In [57]:
length = 100

In [58]:
def get_metascores():
    check = soup.find('span', attrs={'class':'sc-b189961a-1 kcfvgk'})
    ele = soup.find('span', attrs={'class':'sc-b0901df4-0 bcQdDJ metacritic-score-box'})
    for i in range(length):
        if (len(check) == 1):
            metascores.append(np.nan)
        else:
            metascores.append(ele.text)
            ele = ele.find_next('span', attrs={'class':'sc-b0901df4-0 bcQdDJ metacritic-score-box'})
        check = check.find_next('span', attrs={'class':'sc-b189961a-1 kcfvgk'}) 

In [59]:
get_metascores()

In [60]:
# update descriptions 

des_lst = soup.findAll('div', attrs={'class':'ipc-html-content-inner-div'})

descriptions = [des.text for des in des_lst]

In [64]:
print(len(descriptions))

100


In [65]:
movie_DF = pd.DataFrame({'Name of movie': title, 'Description': descriptions, 'Year of relase': years, 'Watchtime': duration,'Classification': classifications, 'Movie Rating': rating, 'Metascore': metascores, 'Votes': vote_count})

In [66]:
movie_DF.to_csv("Top_100_IMDB_Movies.csv", index=False)